# Tarea Dask

## Instrucciones generales
- Esta tarea debe realizarse de manera individual
- Este notebook (resuelto) debe ser subido al github del proyecto en la carpeta de tareas (creen una carpeta dentro de esa carpeta y agreguen su notebook reuelto)
- Fecha límite: Lunes 25 de noviembre de 2024 a las 11:59 p.m
- Deben realizar las cuatro secciones
- Puedes agregar tantas celdas de código y explicaciones como veas necesario, solo manten la estructura general

## Sección 0 Creación y Configuración del cliente de Dask
Ejercicio 0: Configuración del cliente
1. Crea un cliente local de Dask que inicie un clúster en tu máquina.
2. Configura el cliente para que tenga las siguientes características (elige un par de las opciones de trabajadores e hilos):
    - Número de trabajadores: 2 / 4
    - Memoria máxima por trabajador: 1GB
    - Threads por trabajador: 4 / 2
3. Verifica que el cliente esté funcionando correctamente mostrando:
    - Resumen de los trabajadores activos.
    - Dashboard disponible (URL del panel de control de Dask).
    * Tip: Checa los parámetros del cliente que creeaste.

*Nota*: Puedes hacer que corra en el puerto que desees.

In [ ]:
# Tu código va aquí
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=2,               
    threads_per_worker=4,      
    memory_limit='1GB'         
)

client = Client(cluster)

print(client)  

## Sección 1 Delayed
Ejercicio 1: Procesamiento de datos 

1. Genera datos simulados (por ejemplo, ventas diarias) para 10 sucursales durante 365 días.
    - Cada sucursal debe tener datos generados aleatoriamente para "Ingresos" y "Costos".
    - Utiliza una función para generar los datos simulados.
2. Usa Dask Delayed para calcular:
    - Las ganancias diarias por sucursal.
    - La sucursal con mayor ganancia promedio.
3. Genera un grafo de tareas que visualice estas operaciones y explica por qué elegiste paralelizar de esa forma, genera una visualización del grafo.

In [ ]:
# Tu código va aquí
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=2,               
    threads_per_worker=4,      
    memory_limit='1GB'         
)

client = Client(cluster)

print(client)  

## Sección 2 Dask Dataframes
Ejercicio 2: Limpieza y análisis de datos reales

1. Descarga un conjunto de datos masivo (puedes usar la colección de *nycflights* que se encuentra en `data/nycflights/`).
2. Carga los datos en un Dask DataFrame. 
    - Elige adecuadamente el número de particiones (que quepan en memoria de los `workers`)
3. Realiza las siguientes tareas:
    - Limpia los valores faltantes en las columnas `ArrDelay` y `DepDelay`, rellenándolos con la mediana de cada columna.
    - Calcula el retraso promedio (`DepDelay`) por mes y aerolínea.
    - Encuentra el aeropuerto de origen con más vuelos retrasados.

*Nota*: **Evita** convertir el DataFrame a pandas e **intenta** realizar `.compute()` solo cuando sea necesario.

In [ ]:
import dask.dataframe as dd

ruta_datos = "data/nycflights/*.csv"

dtypes = {
    "TailNum": "object",
    "DepDelay": "float64",
    "ArrDelay": "float64",
    "Origin": "object",
    "UniqueCarrier": "object",
}
df = dd.read_csv(ruta_datos, dtype=dtypes, assume_missing=True)

df = df.repartition(npartitions=20) 

mediana_arrdelay = df['ArrDelay'].quantile(0.5).compute()
mediana_depdelay = df['DepDelay'].quantile(0.5).compute()

df = df.fillna({'ArrDelay': mediana_arrdelay, 'DepDelay': mediana_depdelay})

retraso_promedio = (
    df.groupby(['Month', 'UniqueCarrier'])['DepDelay']
    .mean()
    .compute()
    .reset_index()
)

aeropuerto_con_mas_retrasos = (
    df[df['DepDelay'] > 0]
    .groupby('Origin')['DepDelay']
    .count()
    .nlargest(1)
    .compute()
)

print("Retraso promedio (DepDelay) por mes y aerolínea:")
print(retraso_promedio)

print("\nAeropuerto con más vuelos retrasados:")
print(aeropuerto_con_mas_retrasos)

## Sección 3 Dask Arrays

Ejercicio 3: Procesamiento numérico avanzado

1. Crea un arreglo de 10,000 x 10,000 con valores aleatorios usando Dask Array, utiliza un tamaño de chunks adecuado, ¿es mejor que sean cuadrados?.
2. Realiza las siguientes operaciones:
    - Calcula la suma de cada fila.
    - Encuentra la fila con el valor máximo promedio.
    - Multiplica todo el arreglo por un factor escalar (por ejemplo, 2.5).
3. Divide el arreglo nuevamente en 100 bloques y compara la rapidez.

In [ ]:
# Tu código va aquí
import dask.array as da

shape = (10000, 10000)
chunk_size = (1000, 1000)
arr = da.random.random(size=shape, chunks=chunk_size)

suma_filas = arr.sum(axis=1)

promedio_filas = arr.mean(axis=1)
fila_max_promedio = da.argmax(promedio_filas).compute()

arreglo_escalar = arr * 2.5

n_chunks = 100
chunk_size_repartition = (shape[0] // int(n_chunks**0.5), shape[1] // int(n_chunks**0.5))
arr_repartido = arr.rechunk(chunk_size_repartition)

import time

start_original = time.time()
suma_original = arr.sum().compute()
time_original = time.time() - start_original

start_repartido = time.time()
suma_repartido = arr_repartido.sum().compute()
time_repartido = time.time() - start_repartido

print(f"Suma de filas (sin computar): {suma_filas}")
print(f"Fila con el valor máximo promedio: {fila_max_promedio}")
print(f"Suma total del arreglo original: {suma_original}")
print(f"Suma total del arreglo repartido: {suma_repartido}")
print(f"Tiempo para sumar arreglo original: {time_original:.2f} segundos")
print(f"Tiempo para sumar arreglo repartido: {time_repartido:.2f} segundos")

#Elegí chunks de tamaño 1,000 x 1,000 porque equilibran bien el tamaño de los datos procesados y la cantidad de tareas generadas.
#Los chunks cuadrados suelen ser más eficientes porque las operaciones matemáticas distribuyen la carga uniformemente.

## Sección 4 Futures
Ejercicio 4: Distribución de tareas dinámicas

1. Implementa una función que calcule la raíz cuadrada de una lista de 100,000 números enteros generados aleatoriamente.
2. Divide la lista en 10 partes iguales y usa Dask Futures para calcular la raíz cuadrada de cada parte en paralelo.
3. Recolecta los resultados y calcula:
    - El promedio de todos los números procesados.
    - El tiempo total de ejecución (incluyendo envío y recolección de tareas).
4. Observa como se distribuye la carga en el cliente.

*Nota*: en los ejercicios ya vimos como determinar si ya se cumplío una tarea.

In [ ]:
# Tu código va aquí
import time
from math import sqrt

client = Client()

def calcular_raices(lista):
    return [sqrt(x) for x in lista]

n = 100_000
lista = np.random.randint(1, 10_000, size=n).tolist()

particiones = np.array_split(lista, 10)

start_time = time.time()

futures = [client.submit(calcular_raices, part) for part in particiones]

resultados = client.gather(futures)

todos_los_resultados = np.concatenate(resultados)
promedio = np.mean(todos_los_resultados)

tiempo_total = time.time() - start_time

print(f"Promedio de las raíces cuadradas: {promedio:.2f}")
print(f"Tiempo total de ejecución: {tiempo_total:.2f} segundos")

client.close()